In [1]:
import docplex
import pandas as pd
import tensorflow as tf
import numpy as np
import utility
import copy
import mlp_explainer
import mymetrics
import time
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from milp import codify_network
from teste import get_minimal_explanation
from sklearn.metrics import classification_report
import pickle

In [2]:
def load_climate():
    df = pd.read_csv('./datasets/climate_model_simulation_crashes.csv')
    return df
#Glass Dataset
df = load_climate()

scaler = MinMaxScaler()
scaler.fit(df.values[:, :-1])
scaled_df = scaler.transform(df.values[:, :-1])
lower_bound = scaled_df.min()
upper_bound = scaled_df.max()
print(lower_bound, upper_bound)
df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
#targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
targets = df['target'].values
df_scaled['target'] = targets
columns = df_scaled.columns
dataset_name = 'Climate'
result_path = f'{dataset_name}_results'
if not os.path.exists(result_path):
    os.makedirs(result_path)
    print(f"Created directory: {result_path}")
else:
    print(f"Directory already exists: {result_path}")
display(df_scaled)

0.0 1.0000000000000002
Directory already exists: Climate_results


,vconst_corr,vconst_2,vconst_3,vconst_4,vconst_5,vconst_7,ah_corr,ah_bolus,slm_corr,efficiency_factor,tidal_mix_max,vertical_decay_scale,convect_corr,bckgrnd_vdc1,bckgrnd_vdc_ban,bckgrnd_vdc_eq,bckgrnd_vdc_psim,Prandtl,target
0,0.859671,0.928788,0.252421,0.298148,0.169988,0.737921,0.426147,0.568669,0.474011,0.244266,0.103856,0.870925,1.000000,0.447325,0.307274,0.860540,0.797506,0.870159,0
1,0.606367,0.457227,0.359316,0.306302,0.844088,0.937501,0.442487,0.829203,0.295175,0.616383,0.975832,0.916335,0.847149,0.863988,0.346527,0.356977,0.438628,0.512305,1
2,0.998404,0.372474,0.517729,0.505192,0.619230,0.607119,0.745857,0.195983,0.817388,0.679023,0.803377,0.645045,0.719860,0.924776,0.315136,0.250661,0.285677,0.365818,1
3,0.783951,0.102452,0.196926,0.421678,0.742619,0.491993,0.000941,0.392530,0.006825,0.470614,0.597745,0.763119,0.362815,0.912788,0.978782,0.848106,0.699851,0.476013,1
4,0.406331,0.512871,0.060808,0.636602,0.845558,0.442502,0.188403,0.488124,0.357468,0.550894,0.743812,0.312245,0.651382,0.521167,0.042880,0.377137,0.280134,0.132101,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,0.657524,0.488972,0.132920,0.411748,0.087088,0.357003,0.478322,0.029435,0.399290,0.279224,0.383881,0.887841,0.770091,0.458214,0.334277,0.574193,0.610521,0.737892,1
536,0.916598,0.843419,0.519281,0.089032,0.336768,0.896088,0.979657,0.675781,0.261753,0.798072,0.353296,0.043760,0.993356,0.345456,0.512575,0.812605,0.593655,0.142388,0
537,0.478770,0.942190,0.770312,0.952901,0.188910,0.112643,0.745273,0.527745,0.873045,0.191563,0.829540,0.100668,0.549651,0.380490,0.198392,0.869370,0.461834,0.652951,1
538,0.007388,0.779788,0.868822,0.705883,0.984308,0.421232,0.710041,0.174764,0.266066,0.761005,0.436503,0.691343,0.826958,0.981812,0.112638,0.365232,0.201434,0.536599,1


In [3]:
np.random.seed(50)
X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
X = np.concatenate((X_train,X_test),axis=0)
y = np.concatenate((y_train,y_test),axis=0)

training_data = pd.DataFrame(X_train, columns = columns[:-1])
training_data[columns[-1]] = y_train
testing_data = pd.DataFrame(X_test, columns = columns[:-1])
testing_data[columns[-1]] = y_test
dataframe = pd.concat([training_data, testing_data])
data = dataframe.to_numpy()
n_classes = dataframe['target'].nunique()

original_bounds = [[dataframe[dataframe.columns[i]].min(),dataframe[dataframe.columns[i]].max()] for i in range(len(dataframe.columns[:-1]))]
keras_model = tf.keras.models.load_model(f'new_models/{dataset_name}.h5')

In [4]:
original_bounds

[[0.0, 1.0],
 [0.0, 0.9999999999999999],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0000000000000002],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 0.9999999999999999],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0000000000000002],
 [0.0, 1.0],
 [0.0, 0.9999999999999999],
 [0.0, 1.0000000000000002]]

In [5]:
mp_model, output_bounds = codify_network(keras_model, dataframe, 'fischetti', relax_constraints=False)
with open(f'{result_path}/{dataset_name}_mp_model.pkl', 'wb') as f:
    pickle.dump(mp_model, f)

with open(f'bounds/{dataset_name}_output_bounds.pkl', 'wb') as f:
    pickle.dump(output_bounds, f)

In [6]:
predictions = []
possible_classes = np.unique(y_test)
class_indexes = []
class_predictions = []
for i in range(n_classes):
    class_indexes.append([])
    class_predictions.append([])
possible_classes, class_indexes, class_predictions
data = testing_data.to_numpy()
for i in range(len(data)):
    predictions.append(mlp_explainer.model_classification_output(k_model=keras_model, net_input=data[i, :-1])[1].numpy())    
    for j,p_class in enumerate(possible_classes):
        if predictions[-1] == p_class:
            class_indexes[j].append(i)
            class_predictions[j].append(data[i, :-1])
print("Metrics:", classification_report(testing_data.to_numpy()[:, -1], predictions,digits=4))

Metrics:               precision    recall  f1-score   support

         0.0     0.7500    0.0857    0.1538        35
         1.0     0.9202    0.9973    0.9572       370

    accuracy                         0.9185       405
   macro avg     0.8351    0.5415    0.5555       405
weighted avg     0.9055    0.9185    0.8878       405



In [7]:
cols = list(testing_data.columns)
if 'target' not in cols:
    cols.append('target')
predicted_dataset = []
for i,pos_class in enumerate(np.unique(y_test)):
    for instance in (testing_data.to_numpy()[:, :-1][class_indexes[i]]):
        instance = np.append(instance, pos_class.astype('int'))
        predicted_dataset.append(instance)
predicted_dataset = np.asarray(predicted_dataset)
pred_dataset_df = pd.DataFrame(predicted_dataset, columns=cols)
pred_dataset_df['target'] = pred_dataset_df['target'].astype('int')

In [8]:
# Save Bounds to CSV
np.savez(f'bounds/{dataset_name}_data_bounds.npz', original_bounds=original_bounds)
# Save Testing Set to CSV
pred_dataset_df.to_csv(f'{dataset_name}_results/{dataset_name}_X_test.csv', index=False)

In [9]:
metrics_dataframes = []
times_onestep = []
sizes_onestep = []
rsum_onestep = []
coverage_onestep = []
pos_exp_onestep = []
neg_exp_onestep = []
onestep_explanations = []

In [10]:
original_bounds

[[0.0, 1.0],
 [0.0, 0.9999999999999999],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0000000000000002],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 0.9999999999999999],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0000000000000002],
 [0.0, 1.0],
 [0.0, 0.9999999999999999],
 [0.0, 1.0000000000000002]]

In [11]:
def compute_mean_std(arr):
    return np.mean(arr), np.std(arr)

def relative_percentage_diff(new, old):
    if np.any(old == 0):
        print(f'Warning: found possible division by zero')
        return np.where(old != 0, ((new - old) / old) * 100, np.nan)
    return ((new - old) / old) * 100

p_value = 0.25
print(f"p = {p_value}")
times_twostep = []
sizes_twostep = []
rsum_twostep = []
coverage_twostep = []
twostep_explanations = []
for j in range(len(pred_dataset_df['target'].unique())):
    for i, sample in enumerate((testing_data.to_numpy()[:, :-1][class_indexes[j]])):
        start = time.perf_counter()
        
        explanation, minimal = mlp_explainer.run_explanation_doublestep(sample = sample, n_classes=n_classes, kmodel=keras_model, model=mp_model, output_bounds=output_bounds, og_bounds=original_bounds, p=p_value)
        end = time.perf_counter()
        twostep_explanations.append(explanation)
        times_twostep.append(end-start)
        sizes_twostep.append(len(minimal))
        rsum_twostep.append(mymetrics.range_sum(twostep_explanations[-1]))
        
        coverage_twostep.append(len(mymetrics.calculate_coverage(testing_data, twostep_explanations[-1])))

        start = time.perf_counter()
        explanation, minimal = mlp_explainer.run_explanation(sample = sample, n_classes=n_classes, kmodel=keras_model, model=mp_model, output_bounds=output_bounds, og_bounds=original_bounds, enable_log=False,
                                                             )
        end = time.perf_counter()
        onestep_explanations.append(explanation)
        times_onestep.append(end-start)
        sizes_onestep.append(len(minimal))
        rsum_onestep.append(mymetrics.range_sum(onestep_explanations[-1]))
        coverage_onestep.append(len(mymetrics.calculate_coverage(testing_data, onestep_explanations[-1])))
        
times_onestep = np.array(times_onestep)
times_twostep = np.array(times_twostep)
sizes_onestep = np.array(sizes_onestep)
sizes_twostep = np.array(sizes_twostep)
rsum_onestep = np.array(rsum_onestep)
rsum_twostep = np.array(rsum_twostep)
coverage_onestep = np.array(coverage_onestep)
coverage_twostep = np.array(coverage_twostep)

# Compute means and standard deviations
(time_mean_onestep, time_std_onestep) = compute_mean_std(times_onestep)
(time_mean_twostep, time_std_twostep) = compute_mean_std(times_twostep)

(sizes_mean_onestep, sizes_std_onestep) = compute_mean_std(sizes_onestep)
(sizes_mean_twostep, sizes_std_twostep) = compute_mean_std(sizes_twostep)

(rsum_mean_onestep, rsum_std_onestep) = compute_mean_std(rsum_onestep)
(rsum_mean_twostep, rsum_std_twostep) = compute_mean_std(rsum_twostep)

(coverage_mean_onestep, coverage_std_onestep) = compute_mean_std(coverage_onestep)
(coverage_mean_twostep, coverage_std_twostep) = compute_mean_std(coverage_twostep)

# Compute relative percentage differences (Mean & Std)
time_mean_diff = relative_percentage_diff(time_mean_twostep, time_mean_onestep)
sizes_mean_diff = relative_percentage_diff(sizes_mean_twostep, sizes_mean_onestep)
rsum_mean_diff = relative_percentage_diff(rsum_mean_twostep, rsum_mean_onestep)
coverage_mean_diff = relative_percentage_diff(coverage_mean_twostep, coverage_mean_onestep)

time_std_diff = relative_percentage_diff(time_std_twostep, time_std_onestep)
sizes_std_diff = relative_percentage_diff(sizes_std_twostep, sizes_std_onestep)
rsum_std_diff = relative_percentage_diff(rsum_std_twostep, rsum_std_onestep)
coverage_std_diff = relative_percentage_diff(coverage_std_twostep, coverage_std_onestep)

# Compute pointwise relative differences
time_relative_pointwise = relative_percentage_diff(times_twostep, times_onestep)
sizes_relative_pointwise = relative_percentage_diff(sizes_twostep, sizes_onestep)
rsum_relative_pointwise = relative_percentage_diff(rsum_twostep, rsum_onestep)
coverage_relative_pointwise = relative_percentage_diff(coverage_twostep, coverage_onestep)

# Compute pointwise means
time_relative_mean = np.mean(time_relative_pointwise) 
sizes_relative_mean = np.mean(sizes_relative_pointwise)
rsum_relative_mean = np.mean(rsum_relative_pointwise)
coverage_relative_mean = np.mean(coverage_relative_pointwise)

# Compute pointwise standard deviations
time_relative_std = np.std(time_relative_pointwise) 
sizes_relative_std = np.std(sizes_relative_pointwise)
rsum_relative_std = np.std(rsum_relative_pointwise)
coverage_relative_std = np.std(coverage_relative_pointwise)

# Organize Data
all_metrics_data = {
    'Metric': ['Time', 'Size', 'Ranges_Sum', 'Coverage'],
    'ONESTEP_MEAN': [time_mean_onestep, sizes_mean_onestep, rsum_mean_onestep, coverage_mean_onestep],
    'ONESTEP_STD': [time_std_onestep, sizes_std_onestep, rsum_std_onestep, coverage_std_onestep],
    'TWOSTEP_MEAN': [time_mean_twostep, sizes_mean_twostep, rsum_mean_twostep, coverage_mean_twostep],
    'TWOSTEP_STD': [time_std_twostep, sizes_std_twostep, rsum_std_twostep, coverage_std_twostep],
    'MEAN_DIFF_%': [time_mean_diff, sizes_mean_diff, rsum_mean_diff, coverage_mean_diff],
    'STD_DIFF_%': [time_std_diff, sizes_std_diff, rsum_std_diff, coverage_std_diff],
    'POINTWISE_MEAN_%': [time_relative_mean, sizes_relative_mean, rsum_relative_mean, coverage_relative_mean],
    'POINTWISE_STD_%': [time_relative_std, sizes_relative_std, rsum_relative_std, coverage_relative_std]
}
# Display and save
all_metrics_df = pd.DataFrame(all_metrics_data)
display(all_metrics_df)
all_metrics_df.to_csv(f'{result_path}/results_{p_value}.csv', index=False)

#Save Raw Metric Data
raw_df = pd.DataFrame({
    "times_onestep": times_onestep, 
    "times_twostep": times_twostep,
    "sizes_onestep": sizes_onestep, 
    "sizes_twostep": sizes_twostep,
    "rsum_onestep": rsum_onestep, 
    "rsum_twostep": rsum_twostep,
    "coverage_onestep": coverage_onestep, 
    "coverage_twostep": coverage_twostep,
    "time_relative_%": time_relative_pointwise,
    "sizes_relative_%": sizes_relative_pointwise,
    "rsum_relative_%": rsum_relative_pointwise,
    "coverage_relative_%": coverage_relative_pointwise
})

# Save to CSV
raw_df.to_csv(f"{result_path}/raw_metric_data_{p_value}.csv", index=False)

# Save onestep explanations
np.savez(f'{result_path}/onestep_explanations_{p_value}.npz', 
         onestep_explanations=onestep_explanations)

# Save twostep explanations
np.savez(f'{result_path}/twostep_explanations{p_value}.npz', 
         twostep_explanations=twostep_explanations)

p = 0.25


,Metric,ONESTEP_MEAN,ONESTEP_STD,TWOSTEP_MEAN,TWOSTEP_STD,MEAN_DIFF_%,STD_DIFF_%,POINTWISE_MEAN_%,POINTWISE_STD_%
0,Time,0.534172,0.120262,0.810320,0.180841,51.696378,50.372864,53.017449,24.806957
1,Size,10.269136,2.753074,10.269136,2.753074,0.000000,0.000000,0.000000,0.000000
2,Ranges_Sum,12.279118,1.168672,12.307182,1.114568,0.228549,-4.629517,0.371410,4.286480
3,Coverage,1.046914,0.222825,1.049383,0.227777,0.235849,2.222359,0.493827,9.283097
